In [59]:
import Pkg
Pkg.add("FilePathsBase")
using FilePathsBase
rel_path_to_GenX = relpath(joinpath(homedir(), "GenX"))
Pkg.activate("../../../../GenX")
Pkg.status()

   Resolving package versions...
  No Changes to `~/GenX/Project.toml`
  No Changes to `~/GenX/Manifest.toml`


Project GenX v0.4.0
Status `~/GenX/Project.toml`
⌃ [336ed68f] CSV v0.10.14
⌃ [159f3aea] Cairo v1.0.5
  [aaaa29a8] Clustering v0.15.7
  [861a8166] Combinatorics v1.0.2
⌃ [a93c6f00] DataFrames v1.6.1
  [864edb3b] DataStructures v0.18.20
  [7db386b5] DependencyWalker v0.1.2
⌃ [b4f34e82] Distances v0.10.11
  [48062228] FilePathsBase v0.9.22
  [4c0ca9eb] Gtk v1.3.0
⌃ [87dc4568] HiGHS v1.9.0
⌃ [7073ff75] IJulia v1.24.2
⌃ [4076af6c] JuMP v1.22.0
⌃ [b8f27783] MathOptInterface v1.29.0
⌃ [f0f68f2c] PlotlyJS v0.18.13
⌃ [91a5bcdd] Plots v1.40.4
⌃ [731186ca] RecursiveArrayTools v3.18.1
  [3646fa90] ScikitLearn v0.7.0
⌅ [2913bbd2] StatsBase v0.33.21
  [112f6efa] VegaLite v3.3.0
⌃ [ddb6d928] YAML v0.4.9
⌃ [83423d85] Cairo_jll v1.18.0+2
⌃ [36c8627f] Pango_jll v1.52.2+0
  [30392449] Pixman_jll v0.43.4+0
  [ade2ca70] Dates
  [37e2e46d] LinearAlgebra
  [9a3f8284] Random
  [10745b16] Statistics v1.10.0
Info Packages marked with ⌃ and ⌅ have new versions available. Those with ⌃ may be upgradable, but those

  Activating project at `~/GenX`


In [60]:
using JuMP, HiGHS
using Plots
using DataFrames, CSV#, PrettyTables

In [106]:
datadir = joinpath("OPF_data/three_nodes")

gen = CSV.read(joinpath(datadir,"gen.csv"), DataFrame);
gencost = CSV.read(joinpath(datadir,"gencost.csv"), DataFrame);
branch = CSV.read(joinpath(datadir,"branch.csv"), DataFrame);
bus = CSV.read(joinpath(datadir,"bus.csv"), DataFrame);
for f in [gen, gencost, branch, bus]
    rename!(f, lowercase.(names(f)))
end

gen.id = 1:nrow(gen);
gencost.id = 1:nrow(gencost);
branch.id = 1:nrow(branch);
# Reverse direction with same parameters
branch2 = copy(branch)
branch2.f = branch2.fbus
branch2.fbus = branch2.tbus
branch2.tbus = branch2.f
branch2 = branch2[:,names(branch)]
append!(branch,branch2)

#=
If resistance ~= 0 (reactance >> resistance), susceptance = 1 / reactance
https://en.wikipedia.org/wiki/Susceptance#Relationship_to_reactance
=#
branch.sus = 1 ./ branch.x

8-element Vector{Float64}:
 35.587188612099645
 35.587188612099645
 35.587188612099645
 35.587188612099645
 35.587188612099645
 35.587188612099645
 35.587188612099645
 35.587188612099645

In [127]:
# Bus set
bus

Row,bus_i,type,pd,qd,gs,bs,area,vm,va,basekv,zone,vmax,vmin
,Int64,Int64,Int64,Float64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Float64,Float64
1,1,1,0,0.0,0,0,1,1,0,230,1,1.1,0.9
2,2,2,0,0.0,0,0,1,1,0,230,1,1.1,0.9
3,3,2,100,98.61,0,0,1,1,0,230,1,1.1,0.9


In [126]:
# Generator set
gen

Row,bus,pg,qg,qmax,qmin,vg,mbase,status,pmax,pmin,pc1,pc2,qc1min,qc1max,qc2min,qc2max,ramp_agc,ramp_10,ramp_30,ramp_q,apf,id
,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,1,40,0,30,-30,1,100,1,1000,0,0,0,0,0,0,0,0,0,0,0,0,1


In [125]:
# Generator cost set
gencost

Row,model,startup,shutdown,n,x1,y1,id
,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,2,0,0,2,50,0,1


In [124]:
# Transmission lines, with a row for each direction
branch

Row,fbus,tbus,r,x,b,ratea,rateb,ratec,ratio,angle,status,angmin,angmax,id,sus
,Int64,Int64,Float64,Float64,Float64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Float64
1,1,3,0.00281,0.0281,0.00712,100,100,100,0,0,1,-360,360,1,35.5872
2,1,3,0.00281,0.0281,0.00712,100,100,100,0,0,1,-360,360,2,35.5872
3,1,2,0.00281,0.0281,0.00712,100,100,100,0,0,1,-360,360,3,35.5872
4,2,3,0.00281,0.0281,0.00712,100,100,100,0,0,1,-360,360,4,35.5872
5,3,1,0.00281,0.0281,0.00712,100,100,100,0,0,1,-360,360,1,35.5872
6,3,1,0.00281,0.0281,0.00712,100,100,100,0,0,1,-360,360,2,35.5872
7,2,1,0.00281,0.0281,0.00712,100,100,100,0,0,1,-360,360,3,35.5872
8,3,2,0.00281,0.0281,0.00712,100,100,100,0,0,1,-360,360,4,35.5872


In [146]:
#=
Function to solve transport flow problem 
Inputs:
    gen -- dataframe with generator info
    branch -- dataframe with transmission lines info
    gencost -- dataframe with generator info
    bus -- dataframe with bus types and loads
=#
function dc_opf(gen, branch, gencost, bus)
    Transport = Model(HiGHS.Optimizer)
    BaseMVA = bus[:,:basekv][1]
    
    # Sets
    Generators = gen.id
    Nodes = bus.bus_i

    # Decision variables
    @variables(Transport, begin
        GEN[Generators] >= 0
        FLOW[Nodes,Nodes]
        THETA[Nodes]
        LOSSES[Nodes,Nodes]
        AUX_F1[Nodes,Nodes] >=0
        AUX_F2[Nodes,Nodes] >=0
    end)

    # Slack bus with reference angle = 0
    fix(THETA[1],0)
    
    # Objective function
    # Minimize the sum of all generator costs
    @objective(Transport, 
        Min,
        sum(gencost[g,:x1] * GEN[g] for g in Generators)
    )

    # Supply / demand balance constraints with losses
    #    with two non-negative aux variables
    @constraint(Transport, cBalance[i in Nodes],
        sum(GEN[g] for g in gen[gen.bus .== i, :id]) - bus[bus.bus_i .== i, :pd][1] 
        == 
        sum(FLOW[i,j] for j in branch[branch.fbus .== i, :tbus])
        + sum(LOSSES[i,j] for j in branch[branch.fbus .== i, :tbus])
    )

    # Auxilary variable flows
    @constraint(Transport, cLineFlowsAux[l in 1:nrow(branch)],
        FLOW[branch[l,:fbus], branch[l,:tbus]] == 
        AUX_F1[branch[l,:fbus], branch[l,:tbus]]
        - AUX_F2[branch[l,:fbus], branch[l,:tbus]]
    )

    # Max generation constraints
    @constraint(Transport, cMaxGen[g in Generators], GEN[g] <= gen[g,:pmax])

    # Max line flow constraints
    for l in 1:nrow(branch)
        @constraint(Transport, FLOW[branch[l,:fbus],branch[l,:tbus]] <= branch[l, :ratea])
    end
    
    # Flow from phase angle difference
    @constraint(Transport, cLineFlows[l in 1:nrow(branch)],
        FLOW[branch[l,:fbus],branch[l,:tbus]] ==
        BaseMVA*branch[l,:sus]*(THETA[branch[l,:fbus]] - THETA[branch[l,:tbus]])
    )

    # Loss constraint
    @constraint(Transport, cLineLosses[l in 1:nrow(branch)],
        LOSSES[branch[l,:fbus], branch[l,:tbus]] >= 
        branch[l,:r] / BaseMVA # component 1, R / BaseMVA
        * branch[l,:ratea]^2 # component 2, MaxFlow^2
        * ((AUX_F1[branch[l,:fbus],branch[l,:tbus]] + AUX_F2[branch[l,:fbus],branch[l,:tbus]])
            / branch[l,:ratea] - 0.165) # component 3, Flow / MaxFlow - 0.165
    )

    optimize!(Transport)

    # Dataframe of optimal decision variables
    generation = DataFrame(
        id = gen.id,
        node = gen.bus,
        gen = value.(GEN).data
    )

    angles = value.(THETA).data
    flows = DataFrame(
        fbus = branch.fbus,
        tbus = branch.tbus,
        flow = BaseMVA .* branch.sus .* (angles[branch.fbus] - angles[branch.tbus])
    )

    
    losses = DataFrame(
        fbus = branch.fbus,
        tbus = branch.tbus, 
        loss = branch.r ./ BaseMVA .* (branch.ratea.*branch.ratea) .* (abs.(flows.flow) ./ branch.ratea .- 0.165)
    )

    prices = DataFrame(
        node = bus.bus_i,
        value = dual.(cBalance).data
    )

    return (
        generation = generation, 
        angles,
        flows,
        prices,
        losses,
        cost = objective_value(Transport),
        status = termination_status(Transport)
    )
end

dc_opf (generic function with 1 method)

In [147]:
solution = dc_opf(gen, branch, gencost, bus)
solution.generation

Running HiGHS 1.7.0 (git hash: 50670fd4c): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [1e-03, 8e+03]
  Cost   [5e+01, 5e+01]
  Bound  [0e+00, 0e+00]
  RHS    [2e-02, 1e+03]
Presolving model
27 rows, 27 cols, 79 nonzeros  0s
17 rows, 23 cols, 55 nonzeros  0s
17 rows, 23 cols, 55 nonzeros  0s
Presolve : Reductions: rows 17(-19); columns 23(-17); elements 55(-39)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0    -1.4098154045e-03 Pr: 9(475); Du: 0(6.30386e-12) 0s
         17     5.0066070771e+03 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 17
Objective value     :  5.0066070771e+03
HiGHS run time      :          0.00


Row,id,node,gen
,Int64,Int64,Float64
1,1,1,100.132


In [148]:
solution.flows

Row,fbus,tbus,flow
,Int64,Int64,Float64
1,1,3,40.0264
2,1,3,40.0264
3,1,2,20.0175
4,2,3,20.0089
5,3,1,-40.0264
6,3,1,-40.0264
7,2,1,-20.0175
8,3,2,-20.0089


In [149]:
solution.angles

3-element Vector{Float64}:
  0.0
 -0.0024456169561819967
 -0.0048901851153129025

In [150]:
solution.prices

Row,node,value
,Int64,Float64
1,1,50.0
2,2,50.0733
3,3,50.1468


In [151]:
solution.losses

Row,fbus,tbus,loss
,Int64,Int64,Float64
1,1,3,0.0287432
2,1,3,0.0287432
3,1,2,0.00429747
4,2,3,0.00428699
5,3,1,0.0287432
6,3,1,0.0287432
7,2,1,0.00429747
8,3,2,0.00428699
